In [2]:
import time
import threading
import multiprocessing
import itertools
import os
import logging
import random
import string
import requests
from functools import partial
from multiprocessing import Queue
from multiprocessing.pool import ThreadPool

# Definitions

In [3]:
# Process
class Process:
    pid = 0
    prio = 10
    files = []
    sockets = []
    addresses = []
    state = states.READY
    
    def __init__():
        pass
    
    def process_signal(signal):
        if signal == 'SIGSTOP':
            self._stop()
        elif signal == 'SIGCONT':
            self._cont()
        elif signal == 'SIGKILL':
            self._kill()
        ...
            
    def _stop():
        pass
    
    def _cont():
        pass
        
    def _kill():
        pass

NameError: name 'states' is not defined

In [ ]:
# scheduler
for process in processes:
    process.process_signal(SIGCONT)
    # ...
    process.process_signal(SIGSTOP)   

In [ ]:
# Process = threads+
Scheduling is repeated on pthreads level

In [ ]:
class Thread:
    tid = 0
    prio = 10
    state = states.READY
    parent = None
    
    def __init__():
        pass
    
    # ...

In [ ]:
# scheduler
for thread in threads:
    thread.process_signal(SIGCONT)
    # ...
    thread.process_signal(SIGSTOP)   

In [ ]:
# 2 kinds of tasks:
- background execution: 
    - daemon process: OS services
    - multithreaded process: notifications, monitor, watcher, etc
- parallel execution: speedup

In [5]:
Parallel execution can be done in 2 ways:
    - threads
    - processes
    
Which one is better?

# Threads

In [4]:
class timer():
    def __init__(self, message):
        self.message = message

    def __enter__(self):
        self.start = time.time()
        return None

    def __exit__(self, type, value, traceback):
        elapsed_time = (time.time() - self.start)
        print(self.message.format(elapsed_time))

TIME_TO_SLEEP = 1
        
def long_running_task(n=TIME_TO_SLEEP):
    #print(threading.current_thread())
    time.sleep(n)

with timer('Elapsed: {}s'):
#     long_running_task()
    long_running_task()


Elapsed: 1.000542402267456s


In [5]:
with timer('Elapsed: {}s'):
    t1 = threading.Thread(target=long_running_task, args=(TIME_TO_SLEEP/2,))
    t2 = threading.Thread(target=long_running_task, args=(TIME_TO_SLEEP/2,))
    t1.start()
    t2.start()
    #print(threading.current_thread())
    t1.join() # be carefull
    t2.join() 
    
print('next step')
# work with date

Elapsed: 0.500849723815918s
next step


In [6]:
def run_threads(func, data, workers):
    threads = [
        threading.Thread(target=func, args=(data / workers, ))
        for _ in range(workers)
    ]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

In [7]:
workers = 10
DATA_SIZE = 1

with timer('Elapsed: {}s'):
    run_threads(long_running_task, DATA_SIZE, workers)

Elapsed: 0.1357557773590088s


In [100]:
def generate_task_queue(total_tasks, queue_size):
    task_queue = []
    remainder = total_tasks
    while remainder > 0:
        value = min(remainder, random.randint(1, total_tasks//queue_size))
        remainder -= value
        task_queue.append(value)
    return task_queue

assert sum(generate_task_queue(1000, 20)) == 1000
assert sum(generate_task_queue(1000, 1)) == 1000
assert sum(generate_task_queue(1000, 1000)) == 1000
assert sum(generate_task_queue(1000, 42)) == 1000

In [8]:
input_data = [DATA_SIZE / workers for _ in range(workers)]
print(input_data)

with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        
        pool.map(long_running_task, input_data)
#         for chunk in input_data:
#             long_running_task(chunk)

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
[0.347205732259127, 0.08582011443225313, 0.5360061306118391, 0.030968022696780784]
Elapsed: 0.5447676181793213s


<img src="https://www.nginx.com/wp-content/uploads/2016/07/thread-pools-worker-process-event-cycle.png">

# Real world task

In [18]:
def fetch_pic(num_pic):
# def fetch_pic(num_pic, path):
    url = 'https://picsum.photos/400/600'
    path = './pics'
    for _ in range(num_pic):
        random_name = ''.join(random.choices(string.ascii_letters + string.digits, k=5))
        response = requests.get(url)
        if response.status_code == 200:
            with open(f'{path}/{random_name}.jpg', 'wb') as f:
                f.write(response.content)
                #print(f"Fetched pic [{os.getpid()}]: {f.name}")

In [26]:
workers = 80
DATA_SIZE = 100

with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
        # input_data = [(DATA_SIZE // workers, './pics') for _ in range(workers)]
        pool.map(fetch_pic, input_data)

Elapsed: 1.746830940246582s


# IO vs CPU bound tasks

In [59]:
DATA_SIZE = 1_000_000
        

def fill_data(n):
    # print(threading.current_thread())
    while n > 0:
        n -= 1
        lst.append(random.randint(1, 100))
        
lst = []
with timer('Elapsed: {}s'):
#     fill_data(DATA_SIZE, lst)
    fill_data(DATA_SIZE)

Elapsed: 0.6972329616546631s


In [44]:
with timer('Elapsed: {}s'):
#     t1 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2, lst))
#     t2 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2, lst))
    t1 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2,))
    t2 = threading.Thread(target=fill_data, args=(DATA_SIZE // 2,))
    t1.start()
    t2.start()
    t1.join()
    t2.join()

Elapsed: 1.1135923862457275s


In [45]:
lst = []
workers = 16
with timer('Elapsed: {}s'):
    with ThreadPool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
#         pool.map(partial(fill_data, lst=lst), input_data)
        pool.map(fill_data, input_data)
        
print(len(lst), lst[:100])

Elapsed: 1.2437291145324707s
1000000 [74, 8, 10, 69, 9, 37, 34, 67, 84, 64, 35, 89, 69, 6, 47, 36, 1, 57, 36, 32, 93, 90, 21, 73, 72, 19, 57, 34, 12, 37, 19, 55, 86, 50, 90, 36, 24, 55, 15, 4, 49, 33, 2, 77, 9, 65, 46, 41, 56, 37, 90, 35, 45, 74, 73, 60, 9, 75, 14, 11, 47, 26, 83, 27, 98, 19, 95, 27, 26, 21, 54, 35, 28, 38, 48, 39, 92, 32, 6, 28, 30, 17, 42, 39, 90, 47, 48, 22, 61, 15, 59, 77, 45, 41, 50, 91, 60, 66, 35, 24]


In [46]:
lst = []
workers = 16
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = [DATA_SIZE // workers for _ in range(workers)]
#         pool.map(partial(fill_data, lst=lst), input_data)
        pool.map(fill_data, input_data)
        
print(len(lst), lst[:100])

Elapsed: 0.3238530158996582s
0 []


In [103]:
def factorize_naive(n):
    """ A naive factorization method. Take integer 'n', return list of
        factors.
    """
    if n < 2:
        return []
    factors = []
    p = 2

    while True:
        if n == 1:
            return factors

        r = n % p
        if r == 0:
            factors.append(p)
            n = n / p
        elif p * p >= n:
            factors.append(n)
            return factors
        elif p > 2:
            # Advance in steps of 2 over odd numbers
            p += 2
        else:
            # If p == 2, get to 3
            p += 1
    assert False, "unreachable"
    
    return n, factors

In [ ]:
# def factorize_list(lst):
#     result = {}
#     data = [n for n in range(range_number)]
#     for n in lst:
#         result[n] = factorize_naive(n)
#     return result

result = {}
workers = 16
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = pool.map(factorize_naive, input_data)
      result = {
            n:factors
            for n, factors in pool.map(factorize_naive, input_data)
        }
    
print(len(result), result[:100])

In [ ]:
lst = []
workers = 16
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = pool.map(factorize_naive, input_data)
        
print(len(result), lst[:100])

In [104]:
lst = []
workers = 16
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = pool.map(factorize_naive, input_data)
        
print(len(result), lst[:100])

Elapsed: 5.704451322555542s
1000000 [[], [2], [3], [2, 2], [5], [2, 3.0], [7], [2, 2, 2], [3, 3], [2, 5.0], [11], [2, 2, 3.0], [13], [2, 7.0], [3, 5.0], [2, 2, 2, 2], [17], [2, 3, 3], [19], [2, 2, 5.0], [3, 7.0], [2, 11.0], [23], [2, 2, 2, 3.0], [5, 5], [2, 13.0], [3, 3, 3], [2, 2, 7.0], [29], [2, 3, 5.0], [31], [2, 2, 2, 2, 2], [3, 11.0], [2, 17.0], [5, 7.0], [2, 2, 3, 3], [37], [2, 19.0], [3, 13.0], [2, 2, 2, 5.0], [41], [2, 3, 7.0], [43], [2, 2, 11.0], [3, 3, 5.0], [2, 23.0], [47], [2, 2, 2, 2, 3.0], [7, 7], [2, 5, 5], [3, 17.0], [2, 2, 13.0], [53], [2, 3, 3, 3], [5, 11.0], [2, 2, 2, 7.0], [3, 19.0], [2, 29.0], [59], [2, 2, 3, 5.0], [61], [2, 31.0], [3, 3, 7.0], [2, 2, 2, 2, 2, 2], [5, 13.0], [2, 3, 11.0], [67], [2, 2, 17.0], [3, 23.0], [2, 5, 7.0], [71], [2, 2, 2, 3, 3], [73], [2, 37.0], [3, 5, 5], [2, 2, 19.0], [7, 11.0], [2, 3, 13.0], [79], [2, 2, 2, 2, 5.0], [3, 3, 3, 3], [2, 41.0], [83], [2, 2, 3, 7.0], [5, 17.0], [2, 43.0], [3, 29.0], [2, 2, 2, 11.0], [89], [2, 3, 3, 5.0], [7,

In [106]:
lst = []
workers = 16
with timer('Elapsed: {}s'):
    with multiprocessing.Pool(workers) as pool:
        input_data = (i for i in range(1, DATA_SIZE+1))
        result = pool.map(factorize_naive, input_data)
        
print(len(result), lst[:100])

Elapsed: 6.144156217575073s
1000000 [[], [2], [3], [2, 2], [5], [2, 3.0], [7], [2, 2, 2], [3, 3], [2, 5.0], [11], [2, 2, 3.0], [13], [2, 7.0], [3, 5.0], [2, 2, 2, 2], [17], [2, 3, 3], [19], [2, 2, 5.0], [3, 7.0], [2, 11.0], [23], [2, 2, 2, 3.0], [5, 5], [2, 13.0], [3, 3, 3], [2, 2, 7.0], [29], [2, 3, 5.0], [31], [2, 2, 2, 2, 2], [3, 11.0], [2, 17.0], [5, 7.0], [2, 2, 3, 3], [37], [2, 19.0], [3, 13.0], [2, 2, 2, 5.0], [41], [2, 3, 7.0], [43], [2, 2, 11.0], [3, 3, 5.0], [2, 23.0], [47], [2, 2, 2, 2, 3.0], [7, 7], [2, 5, 5], [3, 17.0], [2, 2, 13.0], [53], [2, 3, 3, 3], [5, 11.0], [2, 2, 2, 7.0], [3, 19.0], [2, 29.0], [59], [2, 2, 3, 5.0], [61], [2, 31.0], [3, 3, 7.0], [2, 2, 2, 2, 2, 2], [5, 13.0], [2, 3, 11.0], [67], [2, 2, 17.0], [3, 23.0], [2, 5, 7.0], [71], [2, 2, 2, 3, 3], [73], [2, 37.0], [3, 5, 5], [2, 2, 19.0], [7, 11.0], [2, 3, 13.0], [79], [2, 2, 2, 2, 5.0], [3, 3, 3, 3], [2, 41.0], [83], [2, 2, 3, 7.0], [5, 17.0], [2, 43.0], [3, 29.0], [2, 2, 2, 11.0], [89], [2, 3, 3, 5.0], [7,